In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

import config
from utils.mongo_controller import mongo_controller

In [2]:
def plot_currency_data(df: pd.DataFrame, currency: str = "BOB", _mode: str = "default"):
    """
    Plots either BOB or ARS data from a dataframe that contains
    MongoDB-like nested data. 
    
    If currency == 'BOB', we use:
      - USDT_BOB_Binance (sell_vwap, buy_vwap, sell_volume, buy_volume)
      - USD_BOB_Parallel (sell_price)
      - USD_BOB_Official (hard-coded flat lines for sell=6.96, buy=6.86)

    If currency == 'ARS', we use:
      - USDT_ARS_Binance (sell_vwap, buy_vwap, sell_volume, buy_volume)
      - USDT_ARS_TradingView (open, close, high, low, volume)
      - USD_ARS_Parallel (sell_price)
    
    Missing price data is connected between valid points.
    Missing volume data is left as gaps.
    """

    # Create a figure with a secondary y-axis for volumes
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Ensure 'timestamp' is a datetime (if not already)
    if not pd.api.types.is_datetime64_any_dtype(df['timestamp']):
        df['timestamp'] = pd.to_datetime(df['timestamp'])

    if currency.upper() == "BOB" and _mode == "default":
        start_date = pd.to_datetime("2023-01-01")
        df = df[df['timestamp'] >= start_date]
        #
        # 1) Plot data from USDT_BOB_Binance if present
        #
        if "USDT_BOB_Binance" in df.columns:
            sell_vwap_bob = df["USDT_BOB_Binance"].apply(lambda x: x.get("sell_vwap") if isinstance(x, dict) else None)
            buy_vwap_bob = df["USDT_BOB_Binance"].apply(lambda x: x.get("buy_vwap") if isinstance(x, dict) else None)
            sell_vol_bob = df["USDT_BOB_Binance"].apply(lambda x: x.get("sell_volume") if isinstance(x, dict) else None)
            buy_vol_bob = df["USDT_BOB_Binance"].apply(lambda x: x.get("buy_volume") if isinstance(x, dict) else None)

            # Price traces (connectgaps=True for VWAP)
            fig.add_trace(
                go.Scatter(
                    x=df['timestamp'], y=sell_vwap_bob,
                    mode='lines', name='USDT/BOB Sell Price',
                    connectgaps=True
                ), secondary_y=False
            )
            fig.add_trace(
                go.Scatter(
                    x=df['timestamp'], y=buy_vwap_bob,
                    mode='lines', name='USDT/BOB Buy Price',
                    connectgaps=True
                ), secondary_y=False
            )

            # Volume traces (connectgaps=False for volumes)
            fig.add_trace(
                go.Scatter(
                    x=df['timestamp'], y=sell_vol_bob,
                    mode='lines', name='USDT/BOB Sell Volume',
                    connectgaps=False,
                    line=dict(dash='dot')
                ), secondary_y=True
            )
            fig.add_trace(
                go.Scatter(
                    x=df['timestamp'], y=buy_vol_bob,
                    mode='lines', name='USDT/BOB Buy Volume',
                    connectgaps=False,
                    line=dict(dash='dot')
                ), secondary_y=True
            )

        #
        # 2) Plot data from USD_BOB_Parallel if present
        #
        if "USD_BOB_Parallel" in df.columns:
            sell_price_bob = df["USD_BOB_Parallel"].apply(
                lambda x: x.get("exchange_rate") if isinstance(x, dict) else None)
            fig.add_trace(
                go.Scatter(
                    x=df['timestamp'], y=sell_price_bob,
                    mode='lines', name='USD/BOB Parallel Price',
                    connectgaps=True
                ), secondary_y=False
            )

            #
            # 3) Plot data from USD_BOB_Tarjeta if present
            sell_price_tarjeta = df["USD_BOB_Tarjeta"].apply(
                lambda x: x.get("sell_price") if isinstance(x, dict) else None)
            fig.add_trace(
                go.Scatter(
                    x=df['timestamp'], y=sell_price_tarjeta,
                    mode='lines', name='USD/BOB Tarjeta Price',
                    connectgaps=True
                ), secondary_y=False
            )

        #
        # 4) Add Official BOB lines (fixed)
        #
        official_sell = [6.96] * len(df)
        official_buy = [6.86] * len(df)
        fig.add_trace(
            go.Scatter(
                x=df['timestamp'], y=official_sell,
                mode='lines', name='USD/BOB Official Sell',
                line=dict(dash='dash'),
                connectgaps=True
            ), secondary_y=False
        )
        fig.add_trace(
            go.Scatter(
                x=df['timestamp'], y=official_buy,
                mode='lines', name='USD/BOB Official Buy',
                line=dict(dash='dash'),
                connectgaps=True
            ), secondary_y=False
        )

        title_text = "BOB Data"

    elif currency.upper() == "ARS" and _mode == "default":
        #
        # 1) USDT_ARS_Binance
        #
        if "USDT_ARS_Binance" in df.columns:
            sell_vwap_ars = df["USDT_ARS_Binance"].apply(lambda x: x.get("sell_vwap") if isinstance(x, dict) else None)
            buy_vwap_ars = df["USDT_ARS_Binance"].apply(lambda x: x.get("buy_vwap") if isinstance(x, dict) else None)
            sell_vol_ars = df["USDT_ARS_Binance"].apply(lambda x: x.get("sell_volume") if isinstance(x, dict) else None)
            buy_vol_ars = df["USDT_ARS_Binance"].apply(lambda x: x.get("buy_volume") if isinstance(x, dict) else None)

            # Price traces
            fig.add_trace(
                go.Scatter(
                    x=df['timestamp'], y=sell_vwap_ars,
                    mode='lines', name='USDT/ARS Binance Sell',
                    connectgaps=True
                ), secondary_y=False
            )
            fig.add_trace(
                go.Scatter(
                    x=df['timestamp'], y=buy_vwap_ars,
                    mode='lines', name='USDT/ARS Binance Buy',
                    connectgaps=True
                ), secondary_y=False
            )

            # Volume traces
            fig.add_trace(
                go.Scatter(
                    x=df['timestamp'], y=sell_vol_ars,
                    mode='lines', name='USDT/ARS Binance Sell Volume',
                    connectgaps=False,
                    line=dict(dash='dot')
                ), secondary_y=True
            )
            fig.add_trace(
                go.Scatter(
                    x=df['timestamp'], y=buy_vol_ars,
                    mode='lines', name='USDT/ARS Binance Buy Volume',
                    connectgaps=False,
                    line=dict(dash='dot')
                ), secondary_y=True
            )

        #
        # 2) USDT_ARS_TradingView
        #
        if "USDT_ARS_TradingView" in df.columns:
            close_tv = df["USDT_ARS_TradingView"].apply(lambda x: x.get("close") if isinstance(x, dict) else None)
            vol_tv = df["USDT_ARS_TradingView"].apply(lambda x: x.get("volume") if isinstance(x, dict) else None)

            fig.add_trace(
                go.Scatter(
                    x=df['timestamp'], y=close_tv,
                    mode='lines', name='USDT/ARS Binance/Bitso',
                    connectgaps=True
                ), secondary_y=False
            )
            fig.add_trace(
                go.Scatter(
                    x=df['timestamp'], y=vol_tv,
                    mode='lines', name='USDT/ARS Binance/Bitso Volume',
                    connectgaps=False,
                    line=dict(dash='dot')
                ), secondary_y=True
            )

        #
        # 3) USD_ARS_Parallel
        #
        # In the JSON example, the parallel ARS is under "USD_ARS_Parallel": { "sell_price": ... }
        # so let's plot that as well:
        if "USD_ARS_Parallel" in df.columns:
            parallel_ars = df["USD_ARS_Parallel"].apply(lambda x: x.get("sell_price") if isinstance(x, dict) else None)
            fig.add_trace(
                go.Scatter(
                    x=df['timestamp'], y=parallel_ars,
                    mode='lines', name='USD/ARS Parallel Price',
                    connectgaps=True
                ), secondary_y=False
            )

        #
        # 4) USD_ARS_Official
        #
        if "USD_ARS_Official" in df.columns:
            official_ars = df["USD_ARS_Official"].apply(lambda x: x.get("close") if isinstance(x, dict) else None)
            fig.add_trace(
                go.Scatter(
                    x=df['timestamp'], y=official_ars,
                    mode='lines', name='USD/ARS Official Price',
                    connectgaps=True
                ), secondary_y=False
            )

        title_text = "ARS Data"

    else:  # To clean bad data
        start_date = pd.to_datetime("2023-01-01")
        df = df[df['timestamp'] >= start_date]
        #
        # 1) Plot data from USDT_BOB_Binance if present
        #
        daily_averages_df = mongo_controller.query_data(_mode="all", collection="Daily_Averages", sort=1)
        daily_averages_df = daily_averages_df[daily_averages_df['timestamp'] >= start_date]
        if not pd.api.types.is_datetime64_any_dtype(daily_averages_df['timestamp']):
            daily_averages_df['timestamp'] = pd.to_datetime(daily_averages_df['timestamp'])

        if "USDT_BOB_Binance" in daily_averages_df.columns:
            sell_vwap_bob = daily_averages_df["USDT_BOB_Binance"].apply(
                lambda x: x.get("sell_vwap") if isinstance(x, dict) else None)
            buy_vwap_bob = daily_averages_df["USDT_BOB_Binance"].apply(
                lambda x: x.get("buy_vwap") if isinstance(x, dict) else None)
            sell_vol_bob = daily_averages_df["USDT_BOB_Binance"].apply(
                lambda x: x.get("sell_volume") if isinstance(x, dict) else None)
            buy_vol_bob = daily_averages_df["USDT_BOB_Binance"].apply(
                lambda x: x.get("buy_volume") if isinstance(x, dict) else None)

            # Price traces (connectgaps=True for VWAP)
            fig.add_trace(
                go.Scatter(
                    x=daily_averages_df['timestamp'], y=sell_vwap_bob,
                    mode='lines', name='USDT/BOB Sell Price',
                    connectgaps=True
                ), secondary_y=False
            )
            fig.add_trace(
                go.Scatter(
                    x=daily_averages_df['timestamp'], y=buy_vwap_bob,
                    mode='lines', name='USDT/BOB Buy Price',
                    connectgaps=True
                ), secondary_y=False
            )

            # Volume traces (connectgaps=False for volumes)
            fig.add_trace(
                go.Scatter(
                    x=daily_averages_df['timestamp'], y=sell_vol_bob,
                    mode='lines', name='USDT/BOB Sell Volume',
                    connectgaps=False,
                    line=dict(dash='dot')
                ), secondary_y=True
            )
            fig.add_trace(
                go.Scatter(
                    x=daily_averages_df['timestamp'], y=buy_vol_bob,
                    mode='lines', name='USDT/BOB Buy Volume',
                    connectgaps=False,
                    line=dict(dash='dot')
                ), secondary_y=True
            )

        #
        # 2) Plot data from USD_BOB_Parallel if present
        #
        # Step 1: Aggregate values per day (e.g., take the mean per date)
        global df_daily
        df_daily = df[['timestamp', 'exchange_rate']]
        df_daily = df_daily.groupby(df_daily['timestamp'].dt.date)['exchange_rate'].mean().reset_index()
        df_daily['timestamp'] = pd.to_datetime(df_daily['timestamp'])  # Ensure it's DateTime

        # Step 2: Apply a rolling 3-day average, ignoring missing dates
        # df_daily.set_index('timestamp', inplace=True)
        # df_daily = df_daily.asfreq('D')
        df_daily['3_day_avg'] = df_daily['exchange_rate'].rolling(window=3, min_periods=1).mean()
        # df_daily.reset_index(inplace=True)
        # print(df_daily.head())

        fig.add_trace(
            go.Scatter(
                x=df_daily['timestamp'], y=df_daily['3_day_avg'],
                mode='lines', name='USD/BOB Parallel Price',
                connectgaps=True
            ), secondary_y=False
        )

        title_text = "BOB Data"

    # Figure Layout
    fig.update_layout(
        title=title_text,
        xaxis_title="Timestamp",
        yaxis_title="Price",
        legend=dict(x=0, y=1),
        hovermode="x unified",
        template="plotly_white"
    )
    fig.update_yaxes(title_text="Volume", secondary_y=True)

    # Display the figure in the browser
    pio.show(fig, renderer='browser')
    filename = config.DATA_DIR / f"{currency}_data.html"
    fig.write_html(filename)

In [4]:
daily_averages = mongo_controller.query_data(_mode="all", collection="Daily_Averages", sort=1)
plot_currency_data(daily_averages, "BOB")
# plot_currency_data(bob_parallel, "BOB", _mode="raw")

In [3]:
daily_averages = mongo_controller.query_data(_mode="all", collection="Daily_Averages", sort=1)
plot_currency_data(daily_averages, "ARS")